<center><h1>Guan_Haoyu_HW3</h1></center>
<br>
<br>

Name: Haoyu Guan
<br>
Github Username: ghyharry
<br>
USC ID: 7418934031

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import statistics
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
from scipy.stats import norm 
from scipy.stats import bootstrap 

C:\Users\guanh\anaconda3\envs\myenv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Get the AReM Data Set

In [2]:
path = '../data/AReM/'
folders = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']
dfs = {}
for folder in folders:
    files = glob.glob(path + folder+ '/*csv')
    #print(files)
    thefolder={}
    temp=0
    for file in files:
        temp+=1
        file = file.replace('\\', '/')
        #print(file)
        df = pd.read_csv(file, skiprows=4, index_col='# Columns: time')
        thefolder['dataset'+str(temp)] = df
        
    dfs[folder] = df
#dfs

### (b) Test and Train Data

In [3]:
trainf=[]
testf=['../data/AReM/bending1/dataset1.csv','../data/AReM/bending1/dataset2.csv',
                    '../data/AReM/bending2/dataset1.csv','../data/AReM/bending2/dataset2.csv',
                    '../data/AReM/cycling/dataset1.csv','../data/AReM/cycling/dataset2.csv','../data/AReM/cycling/dataset3.csv',
                    '../data/AReM/lying/dataset1.csv','../data/AReM/lying/dataset2.csv','../data/AReM/lying/dataset3.csv',
                    '../data/AReM/sitting/dataset1.csv','../data/AReM/sitting/dataset2.csv','../data/AReM/sitting/dataset3.csv',
                    '../data/AReM/standing/dataset1.csv','../data/AReM/standing/dataset2.csv','../data/AReM/standing/dataset3.csv',
                    '../data/AReM/walking/dataset1.csv','../data/AReM/walking/dataset2.csv','../data/AReM/walking/dataset3.csv']
for folder in folders:
    files = glob.glob(path + folder+ '/*csv')
    #print(files)
    thefolder={}
    temp=0
    for file in files:
        temp+=1
        file = file.replace('\\', '/')
        if file not in testf:
            trainf.append(file)
        
traindf = pd.concat([pd.read_csv(f, skiprows = 4,index_col='# Columns: time') for f in trainf]) 
testdf = pd.concat([pd.read_csv(f, skiprows = 4,index_col='# Columns: time') for f in testf])
testdf

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
# Columns: time,,,,,,
0,39.25,0.43,22.75,0.43,33.75,1.30
250,39.25,0.43,23.00,0.00,33.00,0.00
500,39.25,0.43,23.25,0.43,33.00,0.00
750,39.50,0.50,23.00,0.71,33.00,0.00
1000,39.50,0.50,24.00,0.00,33.00,0.00
...,...,...,...,...,...,...
118750,36.00,2.45,17.00,5.10,20.50,0.87
119000,34.33,1.89,15.00,2.45,17.00,2.12
119250,33.00,7.35,14.60,3.14,13.00,5.70


### (c) Feature Extraction

#### i. Research

Means, Standard deviations, Skewness, Kurtosis, Maximum, Minimum, Median, Quartile(25%,75%,IQR). 
In my experience, we also need to consider Trend, seasonal and nonseasonal Cycle

#### ii. Extraction

In [4]:
files=trainf+testf

table = []
for file in files:
    df = pd.read_csv(file, skiprows = 4)
    df = df.describe().drop('count').drop(columns = '# Columns: time').T
    df = df[['min','max', 'mean', '50%', 'std', '25%', '75%']]
    #print(df)
    dataset = df.values.reshape(1, -1)
    dataset = pd.DataFrame(dataset)
    table.append(dataset)
features = pd.concat(table)

names = []
for i in range(1, 7):
    names.extend(['min' + str(i), 'max' + str(i), 'mean' + str(i), 'median' + str(i),'standard deviation' + str(i), '1st quartile' + str(i), '3rd quartile' + str(i)])

features.columns = names
#print(features)
features.index = np.arange(1, 89)
features.index.name = 'Instance'

features

,min1,max1,mean1,median1,standard deviation1,1st quartile1,3rd quartile1,min2,max2,mean2,...,standard deviation5,1st quartile5,3rd quartile5,min6,max6,mean6,median6,standard deviation6,1st quartile6,3rd quartile6
Instance,,,,,,,,,,,,,,,,,,,,,
1,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.500,0.0,1.79,0.493292,0.430,0.513506,0.000,0.940
2,33.00,47.75,42.179813,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.330,0.0,2.18,0.613521,0.500,0.524317,0.000,1.000
3,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.250,0.0,1.79,0.383292,0.430,0.389164,0.000,0.500
4,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,2.488862,22.2500,24.000,0.0,5.26,0.679646,0.500,0.622534,0.430,0.870
5,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,3.318301,20.5000,23.750,0.0,2.96,0.555313,0.490,0.487826,0.000,0.830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,35.50,46.25,43.174938,43.670,1.989052,42.50,44.5000,0.0,2.12,0.506583,...,2.983976,12.7500,16.500,0.0,5.72,0.911979,0.830,0.666161,0.470,1.220
85,32.75,47.00,42.760563,44.500,3.398919,41.33,45.3725,0.0,3.34,0.486167,...,4.296574,13.0000,18.565,0.0,5.73,0.842271,0.710,0.722165,0.430,1.090
86,19.33,43.50,34.227771,35.500,4.889576,30.50,37.7500,0.0,14.50,3.995729,...,3.092094,14.7500,18.670,0.0,9.74,3.394125,3.100,1.792090,2.105,4.425


#### iii. Standard Deviation

In [5]:
lower = []
upper = []

for each in features.columns:
    bsr = bs.bootstrap(np.array(features[each]), stat_func = bs_stats.std, alpha = 0.1)
    #res = bootstrap(np.array(features[each]), np.std, confidence_level = 0.9, random_state = 4, method = 'percentile')
    
    lower.append(bsr.lower_bound)
    upper.append(bsr.upper_bound)

stdCI = pd.DataFrame({'std Estimate': features.describe().T['std'], '90% confidence interval lower': lower, '90% confidence interval upper': upper})
stdCI.index.name = 'feature'
stdCI

,std Estimate,90% confidence interval lower,90% confidence interval upper
feature,,,
min1,9.569975,8.278062,10.824563
max1,4.394362,3.478739,5.430721
mean1,5.335718,4.757387,5.935192
median1,5.440054,4.846583,6.043120
standard deviation1,1.772153,1.578328,1.957097
1st quartile1,6.153590,5.623524,6.701932
3rd quartile1,5.138925,4.414486,5.918939
min2,0.000000,0.000000,0.000000
max2,5.062729,4.691208,5.465813


#### iv. Select Features

I will choose means, Standard deviation and 1st or 3rd quartile.
mean: The average value is very important as it reflects the magnitude and approximate center point of the entire data.
Standard deviation: reflect the dispersion, Kurtosis and variance of data.
1st or 3rd quartile: Avoid the influence from outliers, while reflecting to some extent the Skewness.

## 2. ISLR 3.7.4

### (a) Linear Train

We can expect that the training RSS for the cubic regression to be lower than the Training RSS for linear regression. This is because the cubic regression have more degrees of freedom and allows it to overfit the training set with lower training RSS.

### (b) Linear Test

While in test RSS, the linear regression will performe better than cubic regression, since the true relationship is linear, the cubic regression may overfit and have some non-zero beta2 and beta3 that causes higher test RSS.

### (c) Not Linear Train

In most case, the cubic regression would has lower the Training RSS than linear regression. For the similar reason as a), cubic regression is more flexible and allows it to overfit the training set with lower training RSS.

### (d) Not Linear Testing

It can go either way. Since we don't know the true relationship between X and Y, we can't tell if linear regression will have a smaller test RSS or not. There is not enough information and both scenarios are possible.

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 

reference

https://builtin.com/data-science/python-list-files-in-directory
https://stackoverflow.com/questions/40769691/prevent-pandas-read-csv-treating-first-row-as-header-of-column-names
https://stats.stackexchange.com/questions/50807/features-for-time-series-classification
https://www.kaggle.com/code/howoojang/first-kaggle-notebook-following-ts-tutorial/notebook
https://github.com/facebookarchive/bootstrapped/blob/master/examples/bootstrap_intro.ipynb
https://www.researchgate.net/figure/Time-domain-features_tbl1_279991612#:~:text=The%20time%2Ddomain%20features%20are,Kurtosis%20and%20Skewness.%20...&text=View-,...,with%20irrelevant%20or%20redundant%20information.